In [197]:
import numpy as np
from numpy.linalg import norm
from matplotlib import pyplot as plt

from simulation.signal import Array_Signals
from algorithm import *

## 仿真实验


### 声场仿真

设置参数


In [198]:
f = 37500  # 声源频率
sample_interval = 1  # 采样时长
r = 100  # 距离
theta = np.deg2rad(60)  # 角度

d = 0.5
K = 0.5 / d
T = 1  # Cw信号周期
T_on = 10e-3  # Cw信号脉宽
c = 1500  # 声速
fs = 500e3  # 采样频率500k

#### 几何模型

- 阵元1, 2, 3坐标分别为(d1, 0), (d2, 0), (d3, 0), 其中阵元1, 2在x轴负半轴.
- theta为声源S的方位角 (极坐标下角度)

In [199]:
d1, d2, d3 = -(K + 1) * d / 2, -(K - 1) * d / 2, (K + 1) * d / 2
S = np.array([r * np.cos(theta), r * np.sin(theta)])

r1 = float(norm(S - [d1, 0]))
r2 = float(norm(S - [d2, 0]))
r3 = float(norm(S - [d3, 0]))

# 真实时延
t12 = (r1 - r2) / c
t23 = (r2 - r3) / c

#### 生成各阵元电信号AD采样后信号

In [200]:
t = np.arange(0, sample_interval, 1 / fs)
sig = Array_Signals(c, f, T_on, T, S, (d1, d2, d3))
sig.set_params(1, 1, 1, 2, 1e-6)
sig.init_rng(1)
x = sig.next(t, (0, 1))

##### 信号图像

In [201]:
# plt.plot(t, x[0])
# plt.plot(t, x[1])
# plt.plot(t, x[2])
# plt.show()

### 定位

#### 时延估计

In [202]:
x1, x2, x3 = x[:, -int(fs):]
# FIXME: 真的有必要插值吗
tau12_hat = cw_delay_estimation(x1, x2, fs, f, T, T_on, 1)
tau23_hat = cw_delay_estimation(x2, x3, fs, f, T, T_on, 1)

In [203]:
print(f'真实时延差tau12: \t{t12}\n相关时延差tau12\': \t{tau12_hat}\n差异: {np.abs(t12 - tau12_hat) / t12 * 100:.2f}%')
print(f'真实时延差tau23: \t{t23}\n相关时延差tau23\': \t{tau23_hat}\n差异: {np.abs(t23 - tau23_hat) / t23 * 100:.2f}%')

真实时延差tau12: 	0.00016729010515537842
相关时延差tau12': 	0.00011400000000000299
差异: 31.85%
真实时延差tau23: 	0.00016604010320237232
相关时延差tau23': 	0.00016599999999999948
差异: 0.02%


#### 解算方位

In [204]:
r_far, theta_far = far_locate(tau12_hat, tau23_hat, c, K, d)
r_near, theta_near = near_locate(tau12_hat, tau23_hat, c, K, d)
print(f'----真实----\nr: {r:.3f}, theta: {np.rad2deg(theta):.3f}')
print(f'----远场----\nr: {r_far:.3f}, theta: {np.rad2deg(theta_far):.3f}')
print(f'----近场----\nr: {r_near:.3f}, theta: {np.rad2deg(theta_near):.3f}\n方位估计绝对误差: \t{np.rad2deg(theta - theta_near):.3f}度\n方位估计相对误差: \t{(theta - theta_near)/theta * 100:.3f}%')

----真实----
r: 100.000, theta: 60.000
----远场----
r: -2.640, theta: 65.165
----近场----
r: 2.620, theta: 64.770
方位估计绝对误差: 	-4.770度
方位估计相对误差: 	-7.950%
